In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.aer import AerSimulator, AerProvider
from qiskit import IBMQ
from qiskit.compiler import transpile
from time import perf_counter
import numpy as np
from qiskit.algorithms.linear_solvers.hhl import HHL


matrix = np.array([[2/np.sqrt(5), 1/np.sqrt(5)],[1/np.sqrt(5), 2/np.sqrt(5)]])
sol = np.array([np.sqrt(3)/2, 1/2])

qubits = QuantumRegister(4)
measurement = ClassicalRegister(4)
circuit = QuantumCircuit(qubits,measurement)

#t = number of variables + log(2+1/(2*error)) //some magic number
t = 2.344916

#r = number that increases precision of solution as it gets higher, but requires more computation
r = 2
#load sol
circuit.ry(2*np.arccos(matrix[0][0]),qubits[-1])

#phase estimation

#set superposition
circuit.h(qubits[1])
circuit.h(qubits[2])

#trotterization
beta = matrix[0][1]
gamma = (matrix[0][0]-matrix[1][1])/2
iterations = 2**(r-1)

for i in reversed(range(1,len(qubits)-1,-1)):
    for x in range(iterations):
        circuit.crz(((-2*t*gamma*2**(i-1))/2),qubits[i],qubits[-1])
        circuit.crx(((-2*t*beta*2**(i-1))/2),qubits[i],qubits[-1])


circuit.swap(qubits[1],qubits[2])
circuit.h(qubits[2])
#circuit.sdg()